In [12]:
import os
from pprint import pprint
import re
import warnings
from deepdiff import DeepDiff
import openpyxl
import pandas as pd

from compare import append_to_dict, dict_to_table, save_df_to_excel


def light_bom_excel_to_dictionary(filePath: str):
    warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

    # Load the workbook and select the sheet
    wb = openpyxl.load_workbook(filePath, data_only=True)
    ws = wb['Bom']

    # Find the PivotTable named 'BomOracle'
    pivot = None
    for pt in ws._pivots:
        if pt.name == 'BomOracle':
            pivot = pt
            break

    if pivot is None:
        raise ValueError("PivotTable 'BomOracle' not found.")

    # Get the pivot table's displayed range (where it is rendered in the sheet)
    pivot_display_range = pivot.location.ref

    # Extract the displayed data from the range
    data = []
    for row in ws[pivot_display_range]:
        data.append([cell.value for cell in row])

    # Convert to pandas DataFrame
    df = pd.DataFrame(data[1:], columns=data[0])
    df = df[df['Item'].notna()].reset_index(drop=True)
    # Remove suffix pattern from 'Item' column
    df['Item'] = df['Item'].str.replace(r'_\d{2}$', '', regex=True)

    # Rename columns
    df = df.rename(columns={'Supplier_Type': 'SupplyType', 'Item': 'Item name','SE_REVISION':'Revision','Qty':'Quantity' })

    # Keep only columns that are different from the original and not unchanged
    df = df[['SupplyType', 'Item name', 'Revision', 'Quantity', 'Description']]

    # Add 'Depth' column full of 1
    df['Depth'] = 1

    light_bom = {}

    for _, row in df.iterrows():
        item = row['Item name']
        light_bom[item] = row.to_dict()

    return light_bom

bom1 = light_bom_excel_to_dictionary("C:/Users/SESA787052/Downloads/BOM QBOT21000 _rev03.xlsx")
bom2 = light_bom_excel_to_dictionary("C:/Users/SESA787052/Downloads/BOM QBOT21000 _rev03-02.xlsx")

diff = DeepDiff(bom1, bom2, threshold_to_diff_deeper=0)
max_depth = 1

print(bom1)
print(bom2)

item_added = [re.findall(r'\[\'(.*?)\'\]', element.replace('[\'content\']', "").replace("root", ""))
                for element in diff.get('dictionary_item_added', [])]
item_removed = [re.findall(r'\[\'(.*?)\'\]', element.replace('[\'content\']', "").replace("root", ""))
                for element in diff.get('dictionary_item_removed', [])]
item_changed = [(re.findall(r'\[\'(.*?)\'\]', element.replace('[\'content\']', "").replace("root", "")), diff['values_changed'][element])
                for element in diff.get('values_changed', [])]

output = {}

for item in item_added:
    output = append_to_dict(item, bom2, {'type': 'ADDED'}, output)

for item in item_removed:
    output = append_to_dict(item, bom1, {'type': 'REMOVED'}, output)

print(item_changed)

for item in item_changed:
    output = append_to_dict(item[0][:-1], bom1, {'type': 'CHANGED', 'changed_value': item[0][-1], **item[1]}, output)

table_output = dict_to_table(output, max_depth)
columns = ['Level', *[str(i) for i in range(1, max_depth+1)], 'Item', 'Description', 'Revision',
            'Quantity', 'SupplyType', 'ModifyType']

output_df = pd.DataFrame(table_output, columns=columns)
output_path = f"C:\\Users\\{os.getlogin()}\\Downloads\\compare_result.xlsx"


if item_added or item_changed or item_removed:
    save_df_to_excel(output_df, max_depth, output_path)
else:
    print("Nothing changed")

{'7341525': {'SupplyType': None, 'Item name': '7341525', 'Revision': 'XX', 'Quantity': 6.0, 'Description': '7341525 RONDELLES BI-METAL ALU/CU BI25-8 - LOT 100', 'Depth': 1}, '3400208300': {'SupplyType': 'APC', 'Item name': '3400208300', 'Revision': 'XX', 'Quantity': 0.002, 'Description': 'ROLL FOAM PE', 'Depth': 1}, '6716530': {'SupplyType': 'APC', 'Item name': '6716530', 'Revision': 'XX', 'Quantity': 24.0, 'Description': 'SUSPENDER CLIPS M6', 'Depth': 1}, '840-9030': {'SupplyType': 'APC', 'Item name': '840-9030', 'Revision': 'XX', 'Quantity': 1.0, 'Description': 'HANDLE SCHNEIDER GREEN GALAXY VM', 'Depth': 1}, '870-3400120900AF': {'SupplyType': 'APC', 'Item name': '870-3400120900AF', 'Revision': 'XX', 'Quantity': 4.0, 'Description': 'L=1000 - STRUCTURE PROFIL', 'Depth': 1}, '870-3400121100AG': {'SupplyType': 'APC', 'Item name': '870-3400121100AG', 'Revision': 'XX', 'Quantity': 4.0, 'Description': 'L=1800 - VERT STRUCT PROFIL', 'Depth': 1}, '870-3400128900AE': {'SupplyType': 'APC', 'It

In [8]:
import warnings
def find_cost_of(search_string: str):
    """
    Finds the unit cost of an item in a Bill of Materials (BOM) Excel file based on a search string.

    Args:
        search_string (str): The string to search for in the 'Item' column of the BOM.

    Returns:
        pandas.Series: A series containing the unit cost(s) of the item(s) that match the search string.
    """

    filePath = 'C:/Users/SESA787052/Documents/GVXL/BOM Valorisée GVXLIOCQ-QASH01250-PROTO.xlsx'
    
    with warnings.catch_warnings(action="ignore"):
        skip = 0
        BOM = pd.read_excel(filePath, skiprows=skip)
        result_row = BOM[BOM['Item'].str.contains(search_string, na=False)]
        return result_row['Unit Cost']
    


10    67.77
Name: Unit Cost, dtype: float64


In [ ]:
import tkinter as tk
from tkinter import font

def get_text_pixel_width(text, font_name="Calibri", font_size=11):
    root = tk.Tk()
    root.withdraw()  # Hide the main window

    # Create a font object
    tk_font = font.Font(family=font_name, size=font_size)

    # Measure the width of the text
    width = tk_font.measure(text)

    root.destroy()
    return width

# Example usage
if __name__ == "__main__":
    text = " Old : FRONT UPPER/LOWER GVEAC7 CABINED PROTECTION SHIELD"
    font_name = "Calibri"
    font_size = 11

    width = get_text_pixel_width(text, font_name, font_size)
    print(f"The width of the text '{text}' in pixels is: {width}")


The width of the text ' Old : FRONT UPPER/LOWER GVEAC7 CABINED PROTECTION SHIELD' in pixels is: 407
